# 야민정음 데이터셋 생성

## Plan: 야민정음 리뷰 데이터셋 10만 개 증강
train.csv의 깨끗한 리뷰 데이터(약 1.1만 개)를 기반으로, 야민정음.csv의 변환 규칙(시각적 유사성, 한영 혼용 등)을 적용하여 10만 개의 난독화 데이터셋을 생성합니다.

## Steps
1. train.csv에서 원본 데이터 추출
output 컬럼에 있는 깨끗한 리뷰 텍스트(약 11,265개)를 로드하여 'Seed Data'로 사용합니다.

2. 변환 규칙 사전(Dictionary) 정의
야민정음.csv 스타일을 분석하여 매핑 테이블을 만듭니다.
규칙 예시: 대→머, 하→ㅎr, 이→0l, 마→口r 등.

3. Yamin.ipynb 내 생성 함수 구현
def yaminize(text, severity) 함수를 작성합니다.
severity(난이도)에 따라 변환 확률을 조절하여 다양한 난독화 버전을 생성합니다.
10만 개 데이터 생성 및 저장
1.1만 개의 원본 문장을 각각 약 9가지 버전(난이도 상/중/하)으로 증강합니다.
최종 결과를 augmented_reviews.csv로 저장합니다.
Further Considerations
규칙의 퀄리티: 단순 1:1 치환뿐만 아니라 ㅎr 처럼 자모 분리+영문 혼용 규칙도 포함.
데이터 밸런스: 난이도가 너무 높은 데이터만 있으면 학습이 어렵습니다. 그러니 난이도(하) 버전을 섞자.


In [2]:
import pandas as pd
import random
import os

# --- 1. Seed Data 추출 및 저장 ---
if os.path.exists('train.csv'):
    train_df = pd.read_csv('train.csv')
    # train.csv의 output 컬럼이 원본(깨끗한) 데이터입니다.
    # 중복 제거 및 NaN 제거
    clean_reviews = train_df['output'].dropna().unique()
    
    # seed.csv로 저장
    seed_df = pd.DataFrame({'text': clean_reviews})
    seed_df.to_csv('seed.csv', index=False)
    print(f"✅ seed.csv 생성 완료! (데이터 개수: {len(seed_df)}개)")
else:
    print("❌ train.csv가 없습니다. 경로를 확인해주세요.")

✅ seed.csv 생성 완료! (데이터 개수: 11238개)


In [4]:
# --- 2. 야민정음 & 특수문자 매핑 테이블 정의 ---
# 리스트로 정의된 항목은 변환 시 무작위로 하나가 선택됩니다.
yamin_dict = {
    # 1. 시각적 유사성 (Traditional Yamin) & rule.md Update
    '대': ['머', '퍼', '떠'],
    '머': ['대', '퍼', '떠'],
    '퍼': ['대', '머', '떠'],
    '떠': ['대', '머', '퍼'],
    
    '댕': ['멍', '펑', '떵'],
    '멍': ['댕', '펑', '떵'],
    '펑': ['댕', '멍', '떵'],
    '떵': ['댕', '멍', '펑'],
    
    '댸': ['며', '띠', '펴'],
    '며': ['댸', '띠', '펴'],
    '띠': ['댸', '며', '펴'],
    '펴': ['댸', '며', '띠'],
    
    '커': ['귀', '긲'],
    '귀': ['커', '긲'],
    '긲': ['커', '귀'],
    
    '파': ['과', '마', '따'],
    '과': ['파', '마', '따'],
    '마': ['파', '과', '따'],
    '따': ['파', '과', '마'],
    
    '피': ['끠', '괴', '미'],
    '끠': ['피', '괴', '미'],
    '괴': ['피', '끠', '미'],
    '미': ['피', '끠', '괴'],
    
    '비': ['네', 'B', '13'],
    '네': ['비'],
    
    '근': ['ㄹ'],
    'ㄹ': ['근'],
    
    '위': ['읶'],
    '읶': ['위'],
    
    '굿': ['긋'],
    '긋': ['굿'],
    
    '삐': ['볘'],
    '볘': ['삐'],

    '포': ['쪼', '모'],
    '쪼': ['포', '모'],
    '모': ['포', '쪼', '巳'], # 巳 added from original/rule

    '지': ['거', '겨', '저'],
    '거': ['지', '겨', '저'],
    '겨': ['지', '거', '저'],
    '저': ['지', '거', '겨'],
    
    '꺼': ['77ㅓ', '刀ㅓ'],
    
    '공': ['끙'],
    '끙': ['공'],
    
    '길': ['긛'],
    '긛': ['길'],
    
    '돼': ['태', 'EH'],
    '태': ['돼', 'EH'],
    
    '맨': ['괜', '팬', '땐'],
    '괜': ['맨', '팬', '땐'],
    '팬': ['맨', '괜', '땐'],
    '땐': ['맨', '괜', '팬'],
    
    '의': ['익'],
    '익': ['의'],
    
    '루': ['륵'],
    '륵': ['루'],
    
    '빅': ['븨', '넥'],
    '븨': ['빅', '넥'],
    '넥': ['빅', '븨'],
    
    '주': ['즉'],
    '즉': ['주', 'KIГ'],
    
    '쥬': ['즊'],
    '즊': ['쥬'],

    # 2. 기타 한글/모양/특수문자 매핑
    '유': ['윾'],
    '윾': ['유'],
    '우': ['윽'],
    '윽': ['우'],
    '식': ['싀'],
    '싀': ['식'],
    
    '이': ['OI', 'Ol', '01', '0I', '0l', 'ㅇI', 'ㅇl', 'e', '2', 'O1'],
    '병': ['ㅹ'],
    '리': ['21', 'z', 'R1', 'ri'],
    '구': ['ㅋ', 'ヲ', '7'], # 7 added from logic
    '기': ['71'],
    '왕': ['앟', '왐'],
    '왱': ['앻'],
    '욍': ['잏'],
    '방': ['bang'],
    '티': ['더', 'T', 't'],
    '도': ['든', 'ㅌ', 'E', 'do'],
    '또': ['SE'],
    '야': ['OF', 'Oㅑ', 'oㅑ'],
    '백': ['뿌'],
    '무': ['ㅁ'],
    '스': ['^', 's'],
    '사': ['ㅅr', '4', 'sa', 'ㅆ'],
    '응': ['%', '÷'],
    '자': ['ㅈr'],
    '가': ['㉠r', 'Lr', '7r', 'go'],
    '나': ['㈁ㅏ', 'Lㅏ', 'na'],
    '다': ['㉰', 'ㅁ-'],
    '라': ['ㄹr', '2r'],
    '바': ['ㅂr', 'b'],
    '아': ['Oㅏ'],
    '차': ['ㅊr'],
    '카': ['ㅋr'],
    '타': ['ㅌr'],
    '하': ['ㅎr', 'Ha'],
    '냐': ['Lㅑ'],
    '샤': ['ㅅㅑ', '^ㅑ'],
    '니': ['L|', 'L1', 'ni', 'u'],
    '디': ['D', 'ㅁ'], # 디 ↔ ㅁ mentioned
    '시': ['ㅅl', 'tl'],
    '치': ['ㅊl'],
    '키': ['ㅋl', 'key'],
    '히': ['ㅎl'],
    '애': ['OH', 'OII'],
    '개': ['7H'],
    '내': ['Lㅐ', 'LH'],
    '래': ['2H'],
    '재': ['JH', 'H'],
    '에': ['O||'],
    '게': ['7||', 'gay'],
    '세': ['tP', 'say'],
    '요': ['yO', '8', 'dD'],
    '로': ['ro', '2o'],
    '는': ['=','는'],
    '들': ['8', 'dl'],
    '해': ['ㅎri', 'hri', 'ho'],
    '여': ['oㅕ'],
    '음': ['m'],
    '질': ['zl'],
    '읔': ['uk'],
    '와': ['oa', '0a'],
    '소': ['Show', '솓'],
    '고': ['ヱ', '그', '끄'],
    '민': ['괸', '끤'],
    '정': ['경'],
}

# --- 3. 변환 함수 정의 ---
def yaminize(text, prob=0.5):
    """
    text: 원본 문자열
    prob: 각 글자가 변환될 확률 (0.0 ~ 1.0)
    독립 시행: 같은 글자가 여러 번 나와도 각각 독립적으로 변환 여부를 결정합니다.
    예: '멍멍이' -> '댕댕이', '댕멍이', '멍댕이' 등 다양한 조합 가능
    """
    result = []
    for char in text:
        if char in yamin_dict and random.random() < prob:
            replacements = yamin_dict[char]
            if isinstance(replacements, list):
                result.append(random.choice(replacements))
            else:
                result.append(replacements)
        else:
            result.append(char)
    return "".join(result)

# 테스트
sample_text = "여기 진짜 멋진 숙소에요, 풍경도 마음에 들어요."
print(f"원본: {sample_text}")
print(f"변환(50%): {yaminize(sample_text, 0.5)}")
print(f"변환(100%): {yaminize(sample_text, 1.0)}")

# 추가 테스트: 독립 확률 변환 확인 (멍멍이 -> 댕댕이, 댕멍이, 멍댕이 등)
test_word = "멍멍이"
print(f"\n[{test_word}] 변환 테스트 (확률 50%, 5회 반복 - 독립 시행 확인):")
for i in range(5):
    print(f"시도 {i+1}: {yaminize(test_word, 0.5)}")

원본: 여기 진짜 멋진 숙소에요, 풍경도 마음에 들어요.
변환(50%): 여기 진짜 멋진 숙Show에dD, 풍경도 파음O|| 8어yO.
변환(100%): oㅕ71 진짜 멋진 숙솓O||dD, 풍경ㅌ 따mO|| 8어8.

[멍멍이] 변환 테스트 (확률 50%, 5회 반복 - 독립 시행 확인):
시도 1: 떵댕ㅇI
시도 2: 떵펑이
시도 3: 멍펑이
시도 4: 펑댕Ol
시도 5: 펑떵0l


In [5]:
# --- 4. 대량 생성 및 저장 ---
augmented_data = []

# 목표 개수: 약 10만 개
# 시드 데이터: 11,238개 -> 각 문장당 약 9개씩 생성
variants_per_sentence = 9

print(f"🚀 데이터 생성 시작... (총 {len(seed_df) * variants_per_sentence}개 예정)")

for _, row in seed_df.iterrows():
    original = row['text']
    
    # 다양한 난이도로 9개 생성
    # Low severity (0.2 ~ 0.4): 3개 -> 가벼운 변형
    # Medium severity (0.5 ~ 0.7): 3개 -> 중간 변형
    # High severity (0.8 ~ 1.0): 3개 -> 심한 변형
    
    severities = [random.uniform(0.2, 0.4) for _ in range(3)] + \
                 [random.uniform(0.5, 0.7) for _ in range(3)] + \
                 [random.uniform(0.8, 1.0) for _ in range(3)]
                 
    for prob in severities:
        transformed = yaminize(original, prob)
        augmented_data.append({
            'input': transformed,
            'output': original, # 정답 레이블
        })

# DataFrame 생성
augmented_df = pd.DataFrame(augmented_data)

# 셔플
augmented_df = augmented_df.sample(frac=1).reset_index(drop=True)

# 저장 (번역 학습용이므로 input, output 컬럼만 유지)
save_path = 'augmented_reviews.csv'
augmented_df[['input', 'output']].to_csv(save_path, index=False)
print(f"✅ {save_path} 저장 완료! (총 {len(augmented_df)}개)")

# 샘플 확인
print("\n--- 생성된 데이터 샘플 (5개) ---")
print(augmented_df.head())

🚀 데이터 생성 시작... (총 101142개 예정)
✅ augmented_reviews.csv 저장 완료! (총 101142개)

--- 생성된 데이터 샘플 (5개) ---
                                               input  \
0                   그냥 최악. Ero변 소m 어따어마Ha그 화장실E 투명ㅠㅠ   
1  6월O|| 방문했는데, 체크인ho 즉시는 남자분e 너무 불친절했고, 안내ㅌ 이상Ha...   
2             5성급 호텔의 서비s + 해운머 뷰 + 친절함, 좋은 것밖에 없비요.   
3  제7r 부산 ㅅr람이ㄹr 먹고 살자고 서울에 올ㄹroa서 서울 친ㅋ를 만나그 200...   
4                       그럭겨럭 좋았음. 깔끔함. 욕조go 있어서 좋았m.   

                                              output  
0                     그냥 최악. 도로변 소음 어마어마하고 화장실도 투명ㅠㅠ  
1  6월에 방문했는데, 체크인해 주시는 남자분이 너무 불친절했고, 안내도 이상하게 해 ...  
2             5성급 호텔의 서비스 + 해운대 뷰 + 친절함, 좋은 것밖에 없네요.  
3  제가 부산 사람이라 먹고 살자고 서울에 올라와서 서울 친구를 만나고 2006년도부터...  
4                        그럭저럭 좋았음. 깔끔함. 욕조가 있어서 좋았음.  
